# TMDB Data Cleaning & Exploration

This notebook handles the cleaning and preprocessing of raw TMDB movie data.

## Objectives
1. Load raw JSON data
2. Flatten nested JSON columns
3. Clean data types and handle missing values
4. Feature engineering
5. Export cleaned data

In [14]:
import pandas as pd
import numpy as np
import json
import os
from pathlib import Path
import yaml
import ast

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## 1. Load Configuration & Data

Load file paths from config and read raw JSON files.

In [15]:
# Load config
try:
    with open('../config/config.yaml', 'r') as f:
        config = yaml.safe_load(f)
    raw_path = Path('..') / config['paths']['raw_data']
    print(f"Raw data path: {raw_path}")
except FileNotFoundError:
    # Fallback if running from notebook dir without parent context
    raw_path = Path('../data/raw')
    print(f"Config not found, using default path: {raw_path}")

# Load JSON files
data_list = []
if raw_path.exists():
    json_files = list(raw_path.glob('*.json'))
    print(f"Found {len(json_files)} JSON files")
    
    for file in json_files:
        try:
            with open(file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                data_list.append(data)
        except Exception as e:
            print(f"Error reading {file}: {e}")
else:
    print("Raw data directory does not exist!")

df = pd.DataFrame(data_list)
print(f"Initial DataFrame shape: {df.shape}")
df.head()

Raw data path: ..\data\raw
Found 18 JSON files
Initial DataFrame shape: (18, 28)


,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,origin_country,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,credits,keywords
0,False,/u2bZhH3nTf0So0UIC1QxAqBvC07.jpg,"{'id': 386382, 'name': 'Frozen Collection', 'p...",150000000,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...",http://movies.disney.com/frozen,109445,tt2294629,[US],en,Frozen,Young princess Anna of Arendelle dreams about ...,18.2770,/itAKcobTYGpYT8Phwjd8c9hleTo.jpg,"[{'id': 6125, 'logo_path': '/tzsMJBJZINu7GHzrp...","[{'iso_3166_1': 'US', 'name': 'United States o...",2013-11-20,1274219009,102,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Only the act of true love will thaw a frozen h...,Frozen,False,7.249,17189,"{'cast': [{'adult': False, 'gender': 1, 'id': ...","{'keywords': [{'id': 7376, 'name': 'princess'}..."
1,False,/cbcpDn6XJaIGoOil1bKuskU8ds4.jpg,"{'id': 1241, 'name': 'Harry Potter Collection'...",125000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",https://www.warnerbros.com/movies/harry-potter...,12445,tt1201607,[GB],en,Harry Potter and the Deathly Hallows: Part 2,"Harry, Ron and Hermione continue their quest t...",17.3221,/c54HpQmuwXjHq2C9wmoACjxoom3.jpg,"[{'id': 174, 'logo_path': '/ingPVoHnINIrFR7WHm...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",2011-07-12,1341511219,130,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,It all ends.,Harry Potter and the Deathly Hallows: Part 2,False,8.100,21466,"{'cast': [{'adult': False, 'gender': 2, 'id': ...","{'keywords': [{'id': 616, 'name': 'witch'}, {'..."
2,False,/dF6FjTZzRTENfB4R17HDN20jLT2.jpg,"{'id': 328, 'name': 'Jurassic Park Collection'...",150000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",https://www.jurassicworld.com/,135397,tt0369610,[US],en,Jurassic World,Twenty-two years after the events of Jurassic ...,9.3758,/rhr4y79GpxQF9IsfJItRXVaoGs4.jpg,"[{'id': 56, 'logo_path': '/cEaxANEisCqeEoRvODv...","[{'iso_3166_1': 'US', 'name': 'United States o...",2015-06-06,1671537444,124,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The park is open.,Jurassic World,False,6.699,21128,"{'cast': [{'adult': False, 'gender': 2, 'id': ...","{'keywords': [{'id': 2041, 'name': 'island'}, ..."
3,False,/8BTsTfln4jlQrLXUBquXJ0ASQy9.jpg,"{'id': 10, 'name': 'Star Wars Collection', 'po...",245000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.starwars.com/films/star-wars-episod...,140607,tt2488496,[US],en,Star Wars: The Force Awakens,Thirty years after defeating the Galactic Empi...,7.5842,/wqnLdwVXoBjKibFRR5U3y0aDUhs.jpg,"[{'id': 1, 'logo_path': '/tlVSws0RvvtPBwViUyOF...","[{'iso_3166_1': 'US', 'name': 'United States o...",2015-12-15,2068223624,136,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Every generation has a story.,Star Wars: The Force Awakens,False,7.255,20107,"{'cast': [{'adult': False, 'gender': 2, 'id': ...","{'keywords': [{'id': 803, 'name': 'android'}, ..."
4,False,/ehzI1mVcnHqB58NqPyQwpMqcVoz.jpg,"{'id': 9485, 'name': 'The Fast and the Furious...",190000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",https://www.uphe.com/movies/furious-7,168259,tt2820852,[US],en,Furious 7,Deckard Shaw seeks revenge against Dominic Tor...,17.1859,/ktofZ9Htrjiy0P6LEowsDaxd3Ri.jpg,"[{'id': 333, 'logo_path': '/5xUJfzPZ8jWJUDzYtI...","[{'iso_3166_1': 'US', 'name': 'United States o...",2015-04-01,1515400000,139,"[{'english_name': 'Arabic', 'iso_639_1': 'ar',...",Released,Vengeance hits home.,Furious 7,False,7.223,11035,"{'cast': [{'adult': False, 'gender': 2, 'id': ...","{'keywords': [{'id': 830, 'name': 'car race'},..."


## 2. Data Cleaning

### 2.1 Drop Irrelevant Columns

In [16]:
cols_to_drop = ['adult', 'imdb_id', 'original_title', 'video', 'homepage']
existing_cols = [col for col in cols_to_drop if col in df.columns]
df_clean = df.drop(columns=existing_cols).copy()
print(f"Dropped columns: {existing_cols}")
print(f"New shape: {df_clean.shape}")

Dropped columns: ['adult', 'imdb_id', 'original_title', 'video', 'homepage']
New shape: (18, 23)


### 2.2 Flatten Nested Columns

Extract data from: `belongs_to_collection`, `genres`, `production_countries`, `production_companies`, `spoken_languages`.

In [17]:
def extract_name(data):
    """Extract single name from dict."""
    if isinstance(data, dict):
        return data.get('name')
    return np.nan

def extract_names_list(data, key='name', separator='|'):
    """Extract list of names from list of dicts."""
    if isinstance(data, list):
        names = [item.get(key) for item in data if isinstance(item, dict) and item.get(key)]
        return separator.join(names) if names else np.nan
    return np.nan

# Apply extractions
df_clean['collection_name'] = df_clean['belongs_to_collection'].apply(extract_name)
df_clean['genres'] = df_clean['genres'].apply(lambda x: extract_names_list(x))
df_clean['production_countries'] = df_clean['production_countries'].apply(lambda x: extract_names_list(x))
df_clean['production_companies'] = df_clean['production_companies'].apply(lambda x: extract_names_list(x))
df_clean['spoken_languages'] = df_clean['spoken_languages'].apply(lambda x: extract_names_list(x))

# Inspect results
df_clean[['genres', 'collection_name']].head()

,genres,collection_name
0,Animation|Family|Adventure|Fantasy,Frozen Collection
1,Adventure|Fantasy,Harry Potter Collection
2,Action|Adventure|Science Fiction|Thriller,Jurassic Park Collection
3,Adventure|Action|Science Fiction,Star Wars Collection
4,Action|Crime|Thriller,The Fast and the Furious Collection


### 2.3 Handle Missing & Incorrect Data

1. Convert datatypes
2. Replace unrealistic values

In [18]:
# Convert numeric columns
numeric_cols = ['budget', 'id', 'popularity', 'revenue', 'vote_count', 'vote_average', 'runtime']

for col in numeric_cols:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

# Convert release_date
if 'release_date' in df_clean.columns:
    df_clean['release_date'] = pd.to_datetime(df_clean['release_date'], errors='coerce')

# Handle zero values in budget/revenue/runtime
for col in ['budget', 'revenue', 'runtime']:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].replace(0, np.nan)

# Create million USD columns
df_clean['budget_musd'] = df_clean['budget'] / 1_000_000
df_clean['revenue_musd'] = df_clean['revenue'] / 1_000_000

# Handle text placeholders
text_cols = ['overview', 'tagline']
placeholders = ['No Data', 'No Overview', 'n/a', 'nan']
for col in text_cols:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].replace(placeholders, np.nan)

print("Datatypes cleaned.")
df_clean.info()

Datatypes cleaned.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   backdrop_path          18 non-null     object        
 1   belongs_to_collection  16 non-null     object        
 2   budget                 18 non-null     int64         
 3   genres                 18 non-null     object        
 4   id                     18 non-null     int64         
 5   origin_country         18 non-null     object        
 6   original_language      18 non-null     object        
 7   overview               18 non-null     object        
 8   popularity             18 non-null     float64       
 9   poster_path            18 non-null     object        
 10  production_companies   18 non-null     object        
 11  production_countries   18 non-null     object        
 12  release_date           18 non-null     datetime

### 2.4 Filtering

1. Remove duplicates
2. Drop rows with unknown id/title
3. Threshold filtering (at least 10 non-NaN)
4. Status filtering

In [19]:
# Drop duplicates
initial_len = len(df_clean)
df_clean = df_clean.drop_duplicates(subset=['id'], keep='first')

# Drop missing ID/Title
df_clean = df_clean.dropna(subset=['id', 'title'])

# Threshold filtering (keep rows with >= 10 non-nulls)
df_clean = df_clean.dropna(thresh=10)

# Status filtering
if 'status' in df_clean.columns:
    df_clean = df_clean[df_clean['status'] == 'Released']
    df_clean = df_clean.drop(columns=['status'])

print(f"Rows removed: {initial_len - len(df_clean)}")
print(f"Final count: {len(df_clean)}")

Rows removed: 0
Final count: 18


## 3. Feature Engineering

In [20]:
# Extract Cast & Crew info (Basic extraction)
def extract_cast_info(credits_data):
    if isinstance(credits_data, dict):
        cast = credits_data.get('cast', [])
        crew = credits_data.get('crew', [])
        
        # Top 5 cast
        top_cast = [p.get('name') for p in cast[:5]]
        cast_str = '|'.join(top_cast) if top_cast else np.nan
        
        # Director
        director = next((p.get('name') for p in crew if p.get('job') == 'Director'), np.nan)
        
        return pd.Series([cast_str, len(cast), director, len(crew)])
    return pd.Series([np.nan, 0, np.nan, 0])

if 'credits' in df_clean.columns:
    df_clean[['cast', 'cast_size', 'director', 'crew_size']] = df_clean['credits'].apply(extract_cast_info)

# Release Year
df_clean['release_year'] = df_clean['release_date'].dt.year

# Force specific columns to string type
string_cols = ['tagline', 'title', 'collection_name']
for col in string_cols:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].astype(str).replace('nan', np.nan)

### 3.1 Data Anomaly Inspection

In [21]:
# Inspect extracted columns
inspection_cols = ['genres', 'collection_name', 'production_countries', 'production_companies', 'spoken_languages']
for col in inspection_cols:
    if col in df_clean.columns:
        print(f"\n--- Top 10 values for {col} ---")
        print(df_clean[col].value_counts().head(10))


--- Top 10 values for genres ---
genres
Adventure|Action|Science Fiction             3
Action|Adventure|Science Fiction|Thriller    2
Action|Adventure|Science Fiction             2
Adventure|Fantasy                            1
Action|Crime|Thriller                        1
Animation|Family|Adventure|Fantasy           1
Action|Adventure|Fantasy|Science Fiction     1
Science Fiction|Action|Adventure             1
Action|Adventure|Animation|Family            1
Adventure|Science Fiction|Action             1
Name: count, dtype: int64

--- Top 10 values for collection_name ---
collection_name
The Avengers Collection                4
Frozen Collection                      2
Star Wars Collection                   2
Jurassic Park Collection               2
Harry Potter Collection                1
The Fast and the Furious Collection    1
Avatar Collection                      1
The Incredibles Collection             1
Black Panther Collection               1
The Lion King (Reboot) Collection  

## 4. Finalize & Save

In [ ]:
# Reorder columns
desired_order = [
    'id', 'title', 'tagline', 'release_date', 'genres', 'collection_name', 
    'original_language', 'budget_musd', 'revenue_musd', 'production_companies', 
    'production_countries', 'vote_count', 'vote_average', 'popularity', 
    'runtime', 'overview', 'spoken_languages', 'poster_path', 
    'cast', 'cast_size', 'director', 'crew_size'
]

# Select existing columns
final_cols = [c for c in desired_order if c in df_clean.columns]
df_final = df_clean[final_cols].copy()

# Reset index
df_final = df_final.reset_index(drop=True)

print("Final columns:", df_final.columns.tolist())

# Save
processed_path = Path('../data/processed')
processed_path.mkdir(parents=True, exist_ok=True)

output_file = processed_path / 'movies_cleaned.csv'
df_final.to_csv(output_file, index=False)
print(f"Saved to {output_file}")

# Also save as parquet for efficient loading
df_final.to_parquet(processed_path / 'movies_cleaned.parquet', index=False)
print(f"Saved to parquet.")

Final columns: ['id', 'title', 'tagline', 'release_date', 'genres', 'collection_name', 'original_language', 'budget_musd', 'revenue_musd', 'production_companies', 'production_countries', 'vote_count', 'vote_average', 'popularity', 'runtime', 'overview', 'spoken_languages', 'poster_path', 'cast', 'cast_size', 'director', 'crew_size']
Saved to ..\data\processed\movies_cleaned.csv
Saved to parquet.
